In [1]:
%load_ext lab_black
%load_ext autotime
import pandas as pd
import numpy as np

time: 556 ms (started: 2022-09-18 14:34:55 -07:00)


# Hubness

Hubs, a subject I find fascinating. A good place to start on this subject is [Hubs in Space: Popular Nearest Neighbors in High-Dimensional Data](https://www.jmlr.org/papers/v11/radovanovic10a.html). In high-dimensional data, when the k-nearest neighbors are calculated, a tendency emerges for a central point to end up being a neighbor of a lot of other points.

Because there are only k x N edges in the nearest neighbor graph, if one point gets more than its fair share of edges, by necessity other items are getting less. So this also leads to the phenomenon of "anti-hubs", friendless items in the neighbor graph that no-one considers a near neighbor. Of course they still have k neighbors, but they don't appear in the neigbor list of anything else.

A likely outcome for dimensionality reduction methods that can be classified as [neighbor embeddings](https://www.jmlr.org/papers/v23/21-0055.html) is that the hub point appears in many edges and hence gets a disproportionate amount of attention, while an increasing proportion of the dataset are anti-hubs which are effectively outliers. If you were to use the nearest neighbor graph as a search graph, you could never find these anti-hubs by walking the graph, so in that sense they aren't representing your data usefully.

None of this seems like it's good for visualization purposes. In reality, how bad is it? Below is some code to calculate the hubness of various datasets, based on the "k-occurrences" of each item. The k-occurrence just counts the number of times each item appears in the k-nearest neighbor list of other items. From a directed graph perspective, you could also say it's the value of the "out-degree" of the item (count the number edges that point to the item). The hubness of a dataset is the maximum value of any k-occurrence in a dataset. An item is an anti-hub if it has a k-occurrence of 0.

Also, unrelated to hubs, but of interest for neighbor embeddings, I include code to calculate if a given nearest neighbor graph contains more than one component. A graph which is not fully connected can cause a spectral embedding to fail and mean you don't have a good "global" view of the data.

In [2]:
import scipy.sparse
from numba import jit
from scipy.sparse.csgraph import connected_components

import drnb.neighbors as nbnbrs
from drnb.io.dataset import (
    get_dataset_info,
    list_available_datasets,
)


def hubness_summary(n_neighbors, names=None):
    if names is None:
        names = list_available_datasets()
    return hubness_datasets(n_neighbors, names)


def hubness_datasets(n_neighbors, names):
    summaries = []
    for name in names:
        nbrs = None
        try:
            nbrs = get_nbrs(name, n_neighbors)
        except ValueError:
            continue
        ko = k_occurrences(nbrs.idx, n_neighbors=n_neighbors)
        ko_desc = describe(ko)
        ko_desc = pd.concat(
            [ko_desc, pd.Series(get_n_components(nbrs), index=["#comps"])]
        )
        ko_desc.name = name
        summaries.append(ko_desc)
    df = pd.concat(summaries, axis=1).T
    for col_to_norm in ["max", "median"]:
        df[f"n{col_to_norm}"] = df[col_to_norm].astype(float) / n_neighbors
    df["#0%"] = 100.0 * df["#0"] / df["count"]
    int_cols = ["count", "min", "25%", "50%", "75%", "max", "median", "#0", "#comps"]
    df[int_cols] = df[int_cols].astype(np.int32)
    df[["nmax", "nmedian", "#0%"]] = df[["nmax", "nmedian", "#0%"]].applymap(
        "{0:.2f}".format
    )
    dataset_info = get_dataset_info(names)
    df.insert(1, "n_dim", dataset_info.n_dim)
    return df.drop(columns=["mean", "std"]).rename(columns={"count": "n_items"})


def get_nbrs(name, n_neighbors):
    nbrs = nbnbrs.read_neighbors(
        name,
        n_neighbors=n_neighbors + 1,
        exact=True,
    )
    if nbrs is None:
        raise ValueError(f"Couldn't get {n_neighbors} for {name}")
    return nbrs


def k_occurrences(idx, n_neighbors=None, include_self=False):
    if not include_self:
        idx = idx[:, 1:]
    if n_neighbors is None:
        n_neighbors = idx.shape[1]
    if n_neighbors > idx.shape[1]:
        raise ValueError(f"{n_neighbors} > {idx.shape[1]}")

    return _k_occurrences(idx, n_neighbors)


@jit(nopython=True)
def _k_occurrences(idx, n_neighbors):
    n_items = idx.shape[0]
    result = np.zeros(dtype=np.int32, shape=n_items)

    for i in range(n_items):
        for j in range(n_neighbors):
            result[idx[i][j]] += 1

    return result


# https://stackoverflow.com/a/38547818/4096483
def describe(df):
    if isinstance(df, np.ndarray):
        df = pd.Series(df)
    d = df.describe()
    return pd.concat(
        [d, df.agg(["median"]), pd.Series(df[df == 0].count(), index=["#0"])]
    )


def get_n_components(nbrs):
    wadj_graph = nn_to_sparse(nbrs)
    return connected_components(csgraph=wadj_graph, directed=True, return_labels=False)


@jit(nopython=True)
def _nn_to_sparse(idx, dist):
    n_items = idx.shape[0]
    n_neighbors = idx.shape[1]
    size = idx.size
    rows = np.zeros(size, dtype=np.int32)
    cols = np.zeros(size, dtype=np.int32)
    vals = np.zeros(size, dtype=np.float32)

    for i in range(n_items):
        inbrs = i * n_neighbors
        for j in range(n_neighbors):
            idx1d = inbrs + j
            rows[idx1d] = i
            cols[idx1d] = idx[i, j]
            vals[idx1d] = dist[i, j]

    return rows, cols, vals


def nn_to_sparse(nbrs):
    idx = nbrs.idx
    dist = nbrs.dist
    n_items = idx.shape[0]
    rows, cols, vals = _nn_to_sparse(idx, dist)

    return scipy.sparse.coo_matrix((vals, (rows, cols)), shape=(n_items, n_items))

time: 7.2 s (started: 2022-09-18 14:34:55 -07:00)


Please be aware that calculating the k-occurrences (and converting to sparse format, needed for the connected component calculation) is done with numba, otherwise it's rather slow, especially for larger values of `n_neighbors`.

## Before we begin: you need to have some data saved

If you want to run this yourself, you should have some actual datasets save to `DRNB_HOME`. See the example notebooks in `data-pipeline` folder for how to do that.

To warm up the numba JIT and to have something to refer to as I go over the columns, let's run the code for two small datasets with a low `n_neighbors`:

In [3]:
hubness15 = hubness_datasets(15, names=["iris", "s1k"])
hubness15

,n_items,n_dim,min,25%,50%,75%,max,median,#0,#comps,nmax,nmedian,#0%
iris,150,4,0,9,15,20,30,15,1,2,2.00,1.03,0.67
s1k,1000,9,0,7,13,20,69,13,9,1,4.60,0.87,0.90


time: 1.66 s (started: 2022-09-18 14:35:02 -07:00)


Here are what all those numbers mean:

* `n_items`: the number of items in the dataset.
* `n_dim`: the number of features (columns) in the dataset.
* `min`, `25%`, `50%`, `75%`, `max`: these are summary statistics of the k-occurrences, as you get from the pandas `describe` method.
    * `max` is of particular importance because that is the hubness.
    * why is there no `mean`? This is of no interest when it comes to k-occurrences. There are always n_neighbors * N edges in the nearest neighbor graph, so the mean number of k-occurrences will always be n_neighbors.
    * Instead, I have included the median which *is* most definitely of interest.
* `#0`: is the number of items in the dataset with 0 k-occurrence: this is the count of anti-hubs in the dataset.
* `#comps`: the number of connected components in the neighbor graph.
* `nmax`: the same as `max`, but normalized by `n_neighbors`. If the number of edges in the graph were distributed "fairly" this value would be `1`.
* `nmedian`: the normalized version `median`, normalized by `n_neighbors`, just like `nmax`.
* `#0%`: `#0` as a percentage of the dataset, e.g. this one is normalized by `count`. So this is saying what percentage of your dataset are anti-hubs.

### 15 neighbors

So let's look at the 15-neighbor result for some datasets. `n_neighbors=15` is typical for UMAP results.

In [4]:
hubness15 = hubness_summary(15)
hubness15

,n_items,n_dim,min,25%,50%,75%,max,median,#0,#comps,nmax,nmedian,#0%
avonet,11009,11,0,10,15,20,44,15,39,1,2.93,1.00,0.35
cifar10,60000,3072,0,0,2,10,2005,2,18689,1,133.67,0.13,31.15
cifar10act,60000,512,0,9,14,20,67,14,321,1,4.47,0.93,0.54
coil100,7200,49152,1,11,14,18,67,14,0,13,4.47,0.93,0.00
coil20,1440,16384,2,11,14,18,52,14,0,3,3.47,0.93,0.00
fashion,70000,784,0,3,10,21,267,10,6728,1,17.80,0.67,9.61
frey,1965,560,0,9,14,20,45,14,6,1,3.00,0.93,0.31
iris,150,4,0,9,15,20,30,15,1,2,2.00,1.03,0.67
isofaces,698,4096,2,12,15,18,30,15,0,1,2.00,1.00,0.00
isoswiss,20000,3,3,13,15,17,27,15,0,1,1.80,1.00,0.00


time: 8.36 s (started: 2022-09-18 14:35:04 -07:00)


A lot going on here. To get a grip, let's look at what I consider low-dimensional datasets, those with less than 100 dimensions in the initial data:

In [5]:
hubness15.loc[hubness15["n_dim"] < 100]

,n_items,n_dim,min,25%,50%,75%,max,median,#0,#comps,nmax,nmedian,#0%
avonet,11009,11,0,10,15,20,44,15,39,1,2.93,1.00,0.35
iris,150,4,0,9,15,20,30,15,1,2,2.00,1.03,0.67
isoswiss,20000,3,3,13,15,17,27,15,0,1,1.80,1.00,0.00
macosko2015-pca50,44808,50,0,5,11,20,351,11,1018,1,23.40,0.73,2.27
macosko2015z-pca50,44808,50,0,4,9,20,229,9,2407,1,15.27,0.60,5.37
mammoth,50000,3,2,12,15,17,36,15,0,1,2.40,1.00,0.00
penguins,342,4,0,9,14,20,34,14,2,2,2.27,0.93,0.58
s1k,1000,9,0,7,13,20,69,13,9,1,4.60,0.87,0.90
scurvehole,9505,3,4,13,15,17,26,15,0,1,1.73,1.00,0.00
tasic2018-pca50,23822,50,0,8,13,20,94,13,140,5,6.27,0.87,0.59


time: 26.8 ms (started: 2022-09-18 14:35:12 -07:00)


Most of these datasets are pretty well-behaved, with the exception of `macosko2015`, which even in its PCA-reduced forms (Z-scaled or not), tends to form hubs. But for the others, the hubness is quite low. What about high-dimensional datasets? For this, I define high-dimensional as > 1000 dimensions.

In [77]:
hubness15.loc[hubness15["n_dim"] >= 1000]

,n_items,n_dim,min,25%,50%,75%,max,median,#0,#comps,nmax,nmedian,#0%
cifar10,60000,3072,0,0,2,10,2005,2,18689,1,133.67,0.13,31.15
coil100,7200,49152,1,11,14,18,67,14,0,13,4.47,0.93,0.00
coil20,1440,16384,2,11,14,18,52,14,0,3,3.47,0.93,0.00
isofaces,698,4096,2,12,15,18,30,15,0,1,2.00,1.00,0.00
lamanno2020,3850,1999,0,3,9,21,169,9,387,1,11.27,0.60,10.05
macosko2015,44808,3000,0,0,1,4,10542,1,20450,1,702.80,0.07,45.64
macosko2015z,44808,3000,0,0,0,3,25213,0,24275,1,1680.87,0.00,54.18
ng20,18846,3000,0,2,4,9,8260,4,2121,1,550.67,0.27,11.25
norb,48600,9216,0,9,13,19,156,13,1,2,10.40,0.87,0.00
olivetti,400,4096,1,6,11,20,95,11,0,1,6.33,0.73,0.00


time: 10 ms (started: 2022-09-18 16:11:59 -07:00)


Compared to the low-dimensional datasets, these datasets are much more prone to hubness. `macosko2015` is particularly bad, but `tasic2018` (also an scRNAseq dataset), `ng20` (a document dataset) and `cifar10` (an image dataset) are also bad. Also `lamanno2020` (yep, another scRNAseq dataset) is a bit concerning because 10% of the data are anti-hubs. On other hand, `coil20`, `coil100` and `olivetti92x112`, despite being very high-dimensional, don't show that bad a behavior, so simply having a large number of columns isn't enough to indicate hubness. 

In [7]:
hubness50 = hubness_summary(50)
hubness50

,n_items,n_dim,min,25%,50%,75%,max,median,#0,#comps,nmax,nmedian,#0%
avonet,11009,11,0,34,49,65,125,49,9,1,2.50,0.98,0.08
cifar10,60000,3072,0,1,8,36,4261,8,10114,1,85.22,0.16,16.86
cifar10act,60000,512,0,30,46,66,199,46,21,1,3.98,0.92,0.04
coil100,7200,49152,2,26,44,66,295,44,0,5,5.90,0.88,0.00
coil20,1440,16384,4,30,44,68,158,44,0,2,3.16,0.88,0.00
fashion,70000,784,0,14,37,71,759,37,3165,1,15.18,0.74,4.52
frey,1965,560,0,30,46,65,176,46,1,1,3.52,0.92,0.05
iris,150,4,12,42,49,57,95,49,0,1,1.90,0.98,0.00
isofaces,698,4096,7,36,48,62,113,48,0,1,2.26,0.96,0.00
isoswiss,20000,3,21,46,50,54,78,50,0,1,1.56,1.00,0.00


time: 12 s (started: 2022-09-18 14:35:13 -07:00)


In [75]:
hubness50.loc[hubness50["n_dim"] < 100]

,n_items,n_dim,min,25%,50%,75%,max,median,#0,#comps,nmax,nmedian,#0%
avonet,11009,11,0,34,49,65,125,49,9,1,2.50,0.98,0.08
iris,150,4,12,42,49,57,95,49,0,1,1.90,0.98,0.00
isoswiss,20000,3,21,46,50,54,78,50,0,1,1.56,1.00,0.00
macosko2015-pca50,44808,50,0,20,38,65,1132,38,71,1,22.64,0.76,0.16
macosko2015z-pca50,44808,50,0,14,34,66,631,34,385,1,12.62,0.68,0.86
mammoth,50000,3,11,43,50,57,101,50,0,1,2.02,1.00,0.00
penguins,342,4,2,35,49,63,101,49,0,1,2.02,0.99,0.00
s1k,1000,9,2,28,44,66,165,44,0,1,3.30,0.88,0.00
scurvehole,9505,3,17,46,50,55,76,50,0,1,1.52,1.00,0.00
tasic2018-pca50,23822,50,0,30,46,65,222,46,16,2,4.44,0.92,0.07


time: 8.89 ms (started: 2022-09-18 16:11:20 -07:00)


In [78]:
hubness50.loc[hubness50["n_dim"] >= 1000]

,n_items,n_dim,min,25%,50%,75%,max,median,#0,#comps,nmax,nmedian,#0%
cifar10,60000,3072,0,1,8,36,4261,8,10114,1,85.22,0.16,16.86
coil100,7200,49152,2,26,44,66,295,44,0,5,5.90,0.88,0.00
coil20,1440,16384,4,30,44,68,158,44,0,2,3.16,0.88,0.00
isofaces,698,4096,7,36,48,62,113,48,0,1,2.26,0.96,0.00
lamanno2020,3850,1999,0,14,36,71,466,36,85,1,9.32,0.72,2.21
macosko2015,44808,3000,0,0,4,17,18582,4,11609,1,371.64,0.08,25.91
macosko2015z,44808,3000,0,0,2,11,33858,2,16598,1,677.16,0.04,37.04
ng20,18846,3000,0,2,6,14,15957,6,1699,1,319.14,0.12,9.02
norb,48600,9216,1,20,37,64,744,37,0,2,14.88,0.74,0.00
olivetti,400,4096,2,20,39,67,251,39,0,1,5.02,0.78,0.00


time: 10.3 ms (started: 2022-09-18 16:12:04 -07:00)


In [8]:
hubness150 = hubness_summary(150)
hubness150

,n_items,n_dim,min,25%,50%,75%,max,median,#0,#comps,nmax,nmedian,#0%
avonet,11009,11,0,104,146,194,375,146,1,1,2.50,0.97,0.01
cifar10,60000,3072,0,6,29,119,7902,29,4623,1,52.68,0.19,7.71
cifar10act,60000,512,0,88,139,201,553,139,1,1,3.69,0.93,0.00
coil100,7200,49152,2,70,111,200,1119,111,0,1,7.46,0.74,0.00
coil20,1440,16384,4,68,113,203,584,113,0,1,3.89,0.75,0.00
fashion,70000,784,0,48,117,213,1772,117,1601,1,11.81,0.78,2.29
frey,1965,560,2,82,138,205,434,138,0,1,2.89,0.92,0.00
isofaces,698,4096,24,115,157,183,284,157,0,1,1.89,1.05,0.00
isoswiss,20000,3,55,143,151,159,233,151,0,1,1.55,1.01,0.00
kuzushiji,70000,784,0,65,120,200,3056,120,31,1,20.37,0.80,0.04


time: 24.4 s (started: 2022-09-18 14:35:25 -07:00)
